## Web Scraping Etsy, Cleaning the Data, and Uploading the data to AWS S3

In this project, I web scrape Etsy using BeautifulSoup and inspect the HTML elements to find the desired data. Information for product names and prices are found, and added to a DataFrame with Pandas. The data is then cleaned to make it readable and ready for further analysis.

Following this, the DataFrame is converted to CSV and uploaded to an AWS S3 bucket by using the AWS SDK (Boto3) and establishing a connection with the S3 bucket. 

In Amazon Redshift, the data is loaded into a database after setting permisions allowing Redshift to connect with S3. Queries are then performed.

#### Retrieving the HTML content and parsing the data with BeautifulSoup

In [ ]:
# Import necessary libraries

import requests
from bs4 import BeautifulSoup

In [ ]:
# Use the requests library to send a GET request to the URL to retrieve the HTML content

URL = 'https://www.etsy.com/c/home-and-living'
response = requests.get(URL)

In [ ]:
# Save the HTML so multiple GET requests are not needed

scraped_file = "scraped_file.html"
with open(scraped_file, "w", encoding = 'utf-8') as f:
    f.write(response.text)

In [ ]:
# Open the HTML file and parse with Beautiful Soup to create a soup object

soup = BeautifulSoup(open(scraped_file, 'r', encoding = 'utf-8'), 'html.parser')

In [ ]:
# From inspecting the HTML elements, the products are found within h2 tags

h2_tags = soup.find_all('h2')
print(len(h2_tags))
print(h2_tags)

In [ ]:
# The web page shows 64 products, but 69 h2 tags are found. We can see that all the products have an h2 class 
# h2 class="wt-text-caption v2-listing-card__title wt-text-truncate"

product_class = 'wt-text-caption v2-listing-card__title wt-text-truncate'
products = soup.find_all('h2',{'class': product_class})

# We can now see that the correct products have been found

len(products)

In [ ]:
# For finding the product prices, inspecting the element shows that the price is contained within a span tag, specifically
# <span class="currency-value">

price_class = 'currency-value'
prices= soup.find_all('span', {'class': price_class})
print(len(prices))
prices

In [ ]:
# Prices being returned also include the original price before any discount. This is not needed. The real price is found within
# p tag, specifically "wt-text-title-01 lc-price"

# The discounted price is within "wt-text-caption search-collage-promotion-price wt-text-slime wt-text-truncate wt-no-wrap"

original_price_class = 'wt-text-caption search-collage-promotion-price wt-text-slime wt-text-truncate wt-no-wrap'
real_price_class = 'wt-text-title-01 lc-price'
real_prices = soup.find_all('p', {'class': real_price_class})
len(real_prices)

In [ ]:
# 64 prices being returned matches with the number of products, but it still contains the discounted price and original price.
# Using the decompose method from beautiful soup, we can filter out the unneeded information

for p in soup.find_all('p', {'class': original_price_class}):
    p.decompose()

real_prices = soup.find_all('p', {'class': real_price_class})

# Now we can see only the correct prices are showing, the original prices have been discardedb

real_prices

In [ ]:
real_prices = soup.find_all('span', {'class': price_class})
print(len(real_prices))
real_prices

#### Adding the required data to a DataFrame and cleaning it

In [ ]:
# We now have the product names and prices, but they are surrounded by unneeded information. 
# Using the .text method will remove this. Strip will remove the white space

product_names = []
for product in products:
    product_names.append(product.text.strip())
    
product_prices= []
for product in real_prices:
    product_prices.append(product.text.strip())

In [ ]:
import pandas as pd

# Create a dictionary with the product and price data

data = {
    'Product Name': product_names,
    'Price': product_prices
}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

# Convert the 'Price' column to float
df['Price'] = df['Price'].astype(float)

# Replace commas with an empty string in the 'Product Name' column
df['Product Name'] = df['Product Name'].str.replace(',', ' ')

# Display the DataFrame
print(df)

#### Saving the DataFrame as a CSV file and uploading it to S3 bucket

In [ ]:
# Save DataFrame as a csv file

df.to_csv('etsy_products.csv', index=False)

In [ ]:
# Create an S3 client and upload CSV file
import boto3


region_name = 'us-east-1'

# Create an S3 client with the specified region
s3 = boto3.client('s3', region_name=region_name)

bucket = 'web_scraping_projects'
folder = 'Python/Etsy/'

with open('etsy_products.csv', 'rb') as f:
    s3.upload_fileobj(f, bucket, folder + 'etsy_products.csv')